# Laboratorium 2

##### 1. Dane wejściowe:

In [1]:
inputs = ["bbb$",
            "aabbabd",
            "ababcd",
           "abcbccd"]
f = open("1997_714.txt", "r")
inputs.append(f.read())
f.close()

##### 2. Upewnij się że każdy łańcuch kończy się unikalnie (ew. dodając nowy znak)

In [2]:
uniq = "$~`|@#%^&*πœ©ß←↓→þł²³¢≠€½«»ðæŋ’ə…µ" # zakładam, że te znaki starczą

for ix, txt in enumerate(inputs):
#     print(ord(txt[-1]), txt[-1])
    if txt[-1] in txt[:-1]:
        for char in uniq:
            if char not in txt:
                inputs[ix]+=char
                break

for txt in inputs:
    print(f"Ends uniquely? {txt[-1] not in txt[:-1]}")
    
file=inputs[-1]
given=inputs

Ends uniquely? True
Ends uniquely? True
Ends uniquely? True
Ends uniquely? True
Ends uniquely? True


In [3]:
import copy

In [4]:
given = copy.deepcopy(inputs)

##### 3. Zaimplemenuje algorytm konstruujący trie sufiksów

In [5]:
class trienode:
    def __init__(self, label, depth, parent=None, kids=None):
        self.label=label
        self.parent=parent
        self.kids={} if kids==None else kids
        self.link=None
        self.depth=depth
        
    def add(self, child):
        assert child.label not in self.kids
        self.kids[child.label]=child
        return child
    
    def find(self, txt):
        if len(txt)==0:
            return True
        elif txt[0] in self.kids:
            return self.kids[txt[0]].find(txt[1:])
        else:
            return False
        
    def pri(self):
        print(f"Depth: {self.depth} Labeld: {self.label} Kids: ", end="")
        for kid in self.kids:
            print(self.kids[kid].label, end=" ")
        print("\n")
        for kid in self.kids:
            self.kids[kid].pri()

In [6]:
class TRIE:
    def __init__(self, txt):
        self.root=trienode("", 0)
        self.root.link=self.root
        self.make(txt)
    
    def make(self, txt):
        head=self.root
        for i in range(len(txt)):
#             print(head.depth, head.label)
            leaf = self.add(head, txt[head.depth+i:])
            head = self.find(leaf)
            
    def add(self, head, txt):
        leaf=head
#         print(head.depth, head.label, txt)
        for i in range(len(txt)):
            leaf = leaf.add(trienode(txt[i], leaf.depth+1, leaf))
        return leaf
    
    def find(self, sib):
        stack = []
        while sib.link is None: # up
            stack.append(sib.label)
            sib = sib.parent
            
        head = sib.link
        if sib is self.root: # no links on path
            self.root.kids[stack[-1]].link = self.root
            sib = self.root.kids[stack.pop()]
        while len(stack)>0 and stack[-1] in head.kids: # down
            head = head.kids[stack[-1]]
            sib = sib.kids[stack.pop()]
            sib.link = head
        return head
    
    def contains(self, txt):
        return self.root.find(txt)

In [7]:
trie = TRIE("aabbabbc")
print(trie.contains("aabb"), trie.contains("abba"), trie.contains("abbb"), trie.contains("abb"), trie.contains("bbc"), trie.contains("bab"))
# trie.root.pri()

True True False True True True


##### 4. Zaimplementuj alorytm konstruujący drzewo sufiksów w oparciu o algorytm McCreighta

In [8]:
class sufnode:
    def __init__(self, parent=None, frm=0, to=0):
        self.kids = {} # char indexing
        self.parent = parent
        self.link = None
        self.frm = frm
        self.to = to
    
    def add(self, frm, to):
        kid = sufnode(self, frm, to)
        self[suftree.txt[frm]]=kid
        return kid
    
    def split(self, split):
        if split==self.to:
            return
        frm = self.frm
        to = self.to
        self.to = split
        kid = sufnode(self, split,  to)
        kid.kids = self.kids
        self.kids = {}
        self[suftree.txt[split]] = kid
    
    @property
    def length(self):
        return self.to - self.frm
    
    @property
    def label(self):
        return repr(suftree.txt[self.frm:self.to])
    
    def __getitem__(self, ix):
        return self.kids.get(ix)
    
    def __setitem__(self, ix, kid):
        self.kids[ix] = kid
        
    def __str__(self, depth=0):
        return '%s %d %d %s\n%s\n%s'% ("\t"*depth, self.frm, self.to, self.label,
                                       ''.join(key for key in self.kids.keys()),
                                   ''.join(kid.__str__(depth+1) for key, kid in self.kids.items()))

In [9]:
class suftree:
    txt="ananas"
    def __init__(self, txt, simp=False):
        self.root = sufnode(txt)
        suftree.txt = txt
        if simp:
            self.makesimp()
        else:
            self.make()

    def make(self):
        txtlen = len(suftree.txt)
        head = self.root
        leaf = head.add(0, txtlen)
        for i in range(1, txtlen):
#             print(self.root, end="\n\n")
            if head == self.root:
                head, leaf = self.slowscan(head, leaf.frm+1, leaf.to)
            else:
                parent = head.parent
                if parent == self.root:
                    headsib, x = self.fastscan(parent, head.frm+1, head.to)
                else:
                    headsib, x = self.fastscan(parent.link, head.frm, head.to)
                if x < headsib.to: # we need to split
                    headsib.split(x)
                    newhead = headsib
                    leaf = newhead.add(leaf.frm, leaf.to)
                else:
                    newhead, leaf = self.slowscan(headsib, leaf.frm, leaf.to)
                head.link = headsib
                head = newhead
                
    def makesimp(self):
        txtlen = len(suftree.txt)
        head = self.root
        leaf = head.add(0, txtlen)
        for i in range(1, txtlen):
            self.slowscan(head, i, txtlen)
        

    def fastscan(self, node, frm, to):
        length = to - frm
        if length==0:
            return node, node.frm
        tmp = node[suftree.txt[frm]]
        if tmp is None:
            return node, node.frm+length
#             print(frm, to, node.label)
        node=tmp
        while length > node.length:
            frm += node.length
            length -= node.length
            tmp = node[suftree.txt[frm]]
            if tmp is None:
                break
            node=tmp
        return node, node.frm + length
    
    def slowscan(self, node, frm, to):
        if frm == to:
            return node, node
        
        pos = 0
        tmp = node[suftree.txt[frm]]
        if tmp is None:
            leaf = node.add(frm, to)
            return node, leaf
        node = tmp
        
#         print("bef ",frm, to)
        while frm < to:
            if node.frm + pos < node.to:
                if suftree.txt[frm] == suftree.txt[node.frm+pos]:
                    frm += 1
                    pos += 1
                else:
                    break
            else:
                pos = 0
                tmp = node[suftree.txt[frm]]
                if tmp is None:
                    leaf = node.add(frm, to)
                    return node, leaf
                node = tmp
                
        if pos > 0:
            node.split(node.frm+pos)
#         print(frm, to)
        leaf = node.add(frm, to)
        return node, leaf
    
    def contains(self, pattern):
        ix = 0
        head = self.root
        while ix<len(pattern):
            if pattern[ix] in head.kids:
                head = head.kids[pattern[ix]]
                if len(pattern[ix:]) < head.length:
#                     print(suftree.txt[head.frm:head.to], pattern[ix:ix+head.length])
                    return suftree.txt[head.frm:head.frm+len(pattern)-ix] == pattern[ix:]
                if suftree.txt[head.frm:head.to] != pattern[ix:ix+head.length]:
#                     print(suftree.txt[head.frm:head.to], pattern[ix:ix+head.length])
                    return False
                ix += head.length
            else:
                return False
        return True

In [10]:
suf = suftree("aabbabd")
print(suf.contains("bbab"), suf.contains("bbb"))
# print(suf.root)
suf = suftree("aabbabbc")
# print(suf.contains("abba"))
# print(suf.root)
print(suf.contains("aabb"), suf.contains("abba"), suf.contains("abbb"), suf.contains("abb"), suf.contains("bbc"), suf.contains("bab"))


True False
True True False True True True


In [11]:
suf = suftree("ananas")
# print(suf.root)

In [12]:
suf = suftree(inputs[4][200:300] + "œ", True)

In [13]:
print(suf.contains("ustawa"), suf.contains("USTAWA"))

False True


In [14]:
suf = suftree(inputs[4], True)
suf.contains("USTAWA")
# print(suf.root)

True

##### 5. Upewnij się że struktury są poprawne

Poza drobnymi testami bezpośrednio po implementacji zbadam każde podsłowo czy występuje w tekście oraz czy żadne podsłowo z podmienioną literą(nieostatnią) na ostanią, nie występuje w tekście

In [15]:
import random
import string

In [16]:
uniq = "$~`|@#%^&*πœ©ß←↓→þł²³¢≠€½«»ðæŋ’ə…µ" 

def uniquify(txt):
    if txt[-1] in txt[:-1]:
        for char in uniq:
            if char not in txt:
                txt+=char
                break
    return txt

def gensubwords(txt):
    res=[]
    for i in range(len(txt)):
        for j in range(i+1, len(txt)):
            res.append(txt[i:j])
    return res

def mix(words, txt):
    res=[]
    for word in words:
        if len(word)!=1:
            word = list(word)
            word[random.randrange(0, len(word)-1)] = txt[-1]
            word = "".join(word)
            res.append(word)
    return res

def check(true, false, txt):
    trie = TRIE(txt)
    suf = suftree(txt)
    badsuf = suftree(txt, True)
    for i in true:
        if not (trie.contains(i) and suf.contains(i) and badsuf.contains(i)):
            return False
    for i in false:
        if trie.contains(i) or suf.contains(i) or badsuf.contains(i):
            return False
    return True

In [17]:
inputs.pop()#za duża ustawa
inputs.append(''.join(random.choices(string.ascii_uppercase + string.digits, k=100)) + "œ")

In [18]:
inputs.append(''.join(random.choices(string.ascii_uppercase + string.digits, k=300)) + "œ")
inputs.append(''.join(random.choices(string.ascii_uppercase + string.digits, k=500)) + "œ")
for txt in inputs[:]:
    txt=uniquify(txt)
    print(txt)
    res = gensubwords(txt)
    false = mix(res, txt)
    print(check(res, false, txt))

bbb$
True
aabbabd
True
ababcd
True
abcbccd
True
7U21W6T08HA24ZU6RRLO1TKCDFTSG4ZG44HMTJBMQ0BXT06CU6Q5VW64A3HSFWIY0V17LNZFHDXB49EIOEVDMDS33NK9PMVUV6BZœ
True
RUXW2RXPVFPA0PDVGI0P7KV7JDO725MU42KZVM6ZTL4RSLZT5VUPRY691X1TZ82G2P42OMLWUPHHSRAOQFZWSUA485LET28LLK43BZKGC5XAKH3NN2TANCA76EF535Y9OINBJR2ZV76OKK3R6SC3EVYBCPI9OO04XYM80SJR9NXFDJUX8RH480JINZ5TTXLXF5VGFA93CJK4IX9EJA5OV3XSB6O45SZ0PUPLJ1SM2PL7JQ5G8W4DGCUXLS7Y976YQ3YG9O9Y38TT82N0L58F11USQBGX5QTA4YY6HAEWZXZ9œ
True
NPTBQVV4YF77BZALF34OAKZAIVHPWF58F9KILGPPP1F0KASXJUKVJEJCLGZUXGNQ2CS7YCSN72D292300FCBEVK85QHICOGY8VQ6GBQSZO6XRZ3IC3B5210A5ZS6P9WQEK87P8IBFTQXGMHH7G0L3CN40OOKRBXOED5KSLKG8UHH2VAPGJ8WKQ9XLF3R9Y01MG7XQXYM51PX4SVBFGZPGOM80DSAW3SX2NAA1QN1MO74JLMIW9XZDGLSK3SA9XF0EUL3852HELN2LYZILELK7MW6C23PEUSL6YSQJ2KYKDA6WWF8DM1VXCO5RQXQODB8299UCQZOR88OCUEH9D8QKOJ5A4PZSVHAPBUR3XSGB8E3KS77E5GRNXIMJLJMCG42028HYSVQPFWS908VZW7HZHYZ953RM7GD660HFUGWLPLWSR5D0Z887F9GOD02P5A16ZN59RNXOPTXGAD7OQIVKIU47WFIJRE6KSERK0YXK4BL75LLE54Fœ
True


In [19]:
suf = suftree(''.join(random.choices(string.ascii_uppercase + string.digits, k=100000))+"œ")

##### 6.Porównaj szybkość działania konstrukcji struktur dla danych z 1.
W wariantach:\
a) Trie\
b) Drzewo sufiksów bez fast_link i elementów "link"\
c) Drzewo sufiksów z wykorzystaniem procedury fast_find

Trie - działa zbyt wolno i zajmuje zbyt dużo pamięci więc pominiemy konstruowanie dla ustawy

In [20]:
import time

In [21]:
for i in given[:-1]:
    print(f"text with length: {len(i)}")
    %timeit index = TRIE(i)

text with length: 4
20.4 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
text with length: 7
45.8 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
text with length: 6
41.2 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
text with length: 7
47.4 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Drzewo sufiksów bez optymalizacji

In [22]:
for i in given:
    print(f"text with length: {len(i)}")
    %timeit index = suftree(i, True)

text with length: 4
15.7 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
text with length: 7
24.1 µs ± 4.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
text with length: 6
17.6 µs ± 2.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
text with length: 7
18.8 µs ± 535 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
text with length: 246473
4.47 s ± 81.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Drzewo sufiksów z optymalizacją

In [23]:
for i in given:
    print(f"text with length: {len(i)}")
    %timeit index = suftree(i)

text with length: 4
14.2 µs ± 609 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
text with length: 7
23.4 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
text with length: 6
17.3 µs ± 52.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
text with length: 7
20.4 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
text with length: 246473
993 ms ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Jak widać już na tak krótkich testach jest widoczna różnica między Trie a drzewami sufiksów.\
Co więcej, na bardzo długim tekście, jest znacząca różnica między zastosowaniem optymalizacji lub ich brakiem.